In [6]:

import logging
from pathlib import Path

In [74]:
from pathlib import Path
import csv
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import functools
import sys
import torch


pd.options.display.float_format = '{:.3f}'.format

In [75]:
#| export
import logging
logging.basicConfig(
    filename="../datax/logs/logger_bart_sampling.txt",
    filemode='a',
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO
    )

In [8]:
def param_default():
    corpus = 'fm_fc_ms_ff' #<-- Scope
    data_path = Path('/workspaces/code-rationales/semeru-datasets/athena_test/' + corpus + '/')
    data_path_raw = Path(data_path/ 'raw')
    return {
        'bpe_path' : '/workspaces/code-rationales/scripts/universal_tokenizer/universal_tokenizer/roberta_aug_spaces',
        'eval_raw': [data_path_raw / 'eval/input.methods.txt',
                        data_path_raw / 'eval/output.tests.txt'],
        'test_raw': [data_path_raw / 'test/input.methods.txt', 
                        data_path_raw / 'test/output.tests.txt'],
        'train_raw': [data_path_raw / 'train/input.methods.txt', 
                        data_path_raw / 'train/output.tests.txt'],
        'data_labels' : ['test_raw'],#['eval_raw','test_raw','train_raw'], <----- Just Test
        'output_pandas' : data_path / 'pandas/',
        'out_processed' : '/datasets/out_processed/',
        'model_name_or_path' : '/workspaces/code-rationales/data/bart-fairseq/checkpoint_dir_athena_ms/models/', #Model Path
        'checkpoint_file': 'checkpoint_best.pt', #Model
        'output_results' : 'results/' 
    }

In [9]:
#sys.path.clear()

In [10]:
sys.path.append('/workspaces/code-rationales/scripts')

In [11]:
from fairseq.models.transformer import TransformerModel
from tokenizers import ByteLevelBPETokenizer


In [12]:
def load_tokenizer(bpe_path):
    return ByteLevelBPETokenizer(str(bpe_path)+'-vocab.json',str(bpe_path)+'-merges.txt')

In [13]:
def lazy_decode(bpe_java):
    return bpe_java.replace(' ','').replace('Ġ',' ').replace('Ċ','\n')

In [14]:
params = param_default()

### Load Model

In [15]:
#Loading a pretrain model
model = TransformerModel.from_pretrained(
  model_name_or_path = params['model_name_or_path'],
  checkpoint_file = params['checkpoint_file'],
)

2023-07-21 11:13:27 | INFO | fairseq.file_utils | loading archive file /workspaces/code-rationales/data/bart-fairseq/checkpoint_dir_athena_ms/models/
2023-07-21 11:13:28 | INFO | fairseq.tasks.translation | [input.methods] dictionary: 50348 types
2023-07-21 11:13:28 | INFO | fairseq.tasks.translation | [output.tests] dictionary: 50348 types
2023-07-21 11:13:30 | INFO | fairseq.models.fairseq_model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': '/home/davidna/data/dummy/models/checkpoint_dir/tensorboard', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 2, 'fp16_scale_window': 512, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp'

In [ ]:
## Move model to GPU if available and trigger evaluation mode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [20]:
## If GPU busy the use CPU
device = 'cpu'

In [21]:
model = model.to( device ) #WARNING, Verify the device before assigning to memory
model.eval()

GeneratorHubInterface(
  (models): ModuleList(
    (0): BARTModel(
      (encoder): TransformerEncoderBase(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50348, 512, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (fc1): Linear(in_features=512,

### Load Tokenizer

In [16]:
tokenizer = load_tokenizer(params['bpe_path'])

In [17]:
def prettify_java(minified_java):
    "tries to undo Michele's minification. Works decently, although for loops and sets get newlines inserted, and there are no empty lines or comments"
    minified_java = minified_java.replace('{','{\n').replace('}','}\n').replace(';',';\n')
    num_indents = 0
    pretty_java = ''
    for line in minified_java.splitlines():
        if line.lstrip().startswith('}'):
            num_indents -= 1
        pretty_java += num_indents*'    '+line+'\n'
        if line.endswith('{'):
            num_indents += 1
        if line.endswith('}') and not line.lstrip().startswith('}'):
            num_indents -= 1
    return pretty_java

## Data loading

In [18]:
#export
def method_size_vector( method_vector ):
    '''Return the size of the tokens for a give method based on id
        Assuming that method_vector is an array of tokens
    '''
    input_ids = [ len(mtd) for mtd in method_vector ]
    return input_ids

In [30]:
def super_set_code():
    '''This method creates a dictionary with as many dataframes as data labels exists, for isntance, input and uotput for each 
    type: test,eval, val'''
    data = {}
    for label in params['data_labels']:
        for val,path_data in enumerate(params[ label ]):
            print(path_data)
            df = pd.read_csv( path_data, sep="\0", header=None, names=[label+str(val)]) #reading file
            df[label+'_bpe'+str(val)] = [ enc.tokens for enc in tokenizer.encode_batch( df[label+str(val)].values ) ] #bpe
            df['method_size'+str(val)] = method_size_vector( df[label+'_bpe'+str(val)].values ) #counting tokens
            data[label+str(val)] =  df  
    return data

In [28]:
super_data = super_set_code()

/workspaces/code-rationales/semeru-datasets/athena_test/fm_fc_ms_ff/raw/test/input.methods.txt
/workspaces/code-rationales/semeru-datasets/athena_test/fm_fc_ms_ff/raw/test/output.tests.txt


In [31]:
## the outoput dataset
super_data['test_raw1']

,test_raw1,test_raw_bpe1,method_size1
0,@Test public void yearStart() { Date date = Da...,"[@, Test, Ġpublic, Ġvoid, Ġyear, Start, (), Ġ{...",61
1,@Test public void yearEnd() { Date date = Date...,"[@, Test, Ġpublic, Ġvoid, Ġyear, End, (), Ġ{, ...",74
2,@Test void validate_invalid_bindingId_test() {...,"[@, Test, Ġvoid, Ġvalidate, _, in, valid, _, b...",110
3,@Test void getUnsignedShort_test1() { byte[] b...,"[@, Test, Ġvoid, Ġget, Un, signed, Short, _, t...",73
4,@Test void isWithinUnsignedLong_test() { asser...,"[@, Test, Ġvoid, Ġis, Within, Un, signed, Long...",145
...,...,...,...
78383,@Test public void testWithBushData() throws IO...,"[@, Test, Ġpublic, Ġvoid, Ġtest, With, Bush, D...",92
78384,@Test public void column() { TableSlice slice ...,"[@, Test, Ġpublic, Ġvoid, Ġcolumn, (), Ġ{, ĠTa...",137
78385,@Test void testIsIn() { StringColumn stringCol...,"[@, Test, Ġvoid, Ġtest, Is, In, (), Ġ{, ĠStrin...",91
78386,@Test public void rightJoinStudentInstructorOn...,"[@, Test, Ġpublic, Ġvoid, Ġright, Join, Studen...",85


In [79]:
super_data['test_raw1'].describe()

,method_size1
count,78388.000
mean,159.106
std,450.521
min,7.000
25%,61.000
50%,102.000
75%,179.000
max,62933.000


In [32]:
super_data['test_raw0']

,test_raw0,test_raw_bpe0,method_size0
0,DateUtils { public static Date yearStart() { f...,"[Date, Ut, ils, Ġ{, Ġpublic, Ġstatic, ĠDate, Ġ...",227
1,DateUtils { public static Date yearEnd() { fin...,"[Date, Ut, ils, Ġ{, Ġpublic, Ġstatic, ĠDate, Ġ...",250
2,TokenBindingValidator { public void validate(T...,"[Token, B, inding, Valid, ator, Ġ{, Ġpublic, Ġ...",197
3,UnsignedNumberUtil { public static int getUnsi...,"[Un, signed, Number, Ut, il, Ġ{, Ġpublic, Ġsta...",239
4,UnsignedNumberUtil { public static boolean isW...,"[Un, signed, Number, Ut, il, Ġ{, Ġpublic, Ġsta...",251
...,...,...,...
78383,CsvReader extends FileReader implements DataRe...,"[C, sv, Reader, Ġextends, ĠFile, Reader, Ġimpl...",80
78384,TableSlice extends Relation { @Override public...,"[Table, Sl, ice, Ġextends, ĠRel, ation, Ġ{, Ġ@...",281
78385,StringColumn extends AbstractStringColumn<Stri...,"[String, Column, Ġextends, ĠAbstract, String, ...",605
78386,DataFrameJoiner { public Table rightOuter(Tabl...,"[Data, Frame, Jo, iner, Ġ{, Ġpublic, ĠTable, Ġ...",329


In [34]:
MAX_TOKENS = 512 ## HARDCODED <-----
def joining_encode_tokens( arr_tokens, model ):
    if len(arr_tokens) > MAX_TOKENS:
        arr_tokens = arr_tokens[0:MAX_TOKENS]
    focal_code = " ".join(arr_tokens)
    return model.encode( focal_code )

## Just Naive Sampling

In [36]:
FLAG_SAMPLING = True #<---- HARDCODED
if FLAG_SAMPLING:
    SIZE_SAMPLING = 100 #<---- HARDCODED
    input_tokens = [ joining_encode_tokens(arr_sample, model=model) for arr_sample in super_data['test_raw0'].test_raw_bpe0.values[:SIZE_SAMPLING] ]
else:
    input_tokens = [ joining_encode_tokens(arr_sample, model=model) for arr_sample in super_data['test_raw0'].test_raw_bpe0.values]

In [37]:
len(input_tokens)

100

In [40]:
SAMPLES = 1 #<---- Hardocoded
MAX_GEN_TOK = 100

In [41]:
def sample_generation(input, model):
    generated_input =  model.generate( 
        input, 
        maxlen=MAX_GEN_TOK, ##WARNING, This parameter is not working
        max_length=SAMPLES, 
        do_sample=False, 
        pad_token_id=50256 ) ## HARDCODED
    return generated_input[0]['tokens'][:MAX_GEN_TOK]

In [42]:
arr_generated_input = [ sample_generation(input, model=model) for input in input_tokens ]

In [43]:
decoded = model.decode(arr_generated_input[0])
decoded

'@ Test Ġpublic Ġvoid Ġtest Year Start () Ġthrows ĠException Ġ{ ĠDate Ġdate Ġ= ĠDate Ut ils . year Start (); Ġassert Not Null ( date ); Ġassert Equ als ( date . get Time (), Ġdate . get Time ()); Ġassert Equ als ( date . get Time (), Ġdate . get Time ()); Ġ}'

In [44]:
prettify_java( lazy_decode( decoded ) )

'@Test public void testYearStart() throws Exception {\n     Date date = DateUtils.yearStart();\n     assertNotNull(date);\n     assertEquals(date.getTime(), date.getTime());\n     assertEquals(date.getTime(), date.getTime());\n }\n'

In [57]:
super_data['test_raw1'].test_raw1.values[:SIZE_SAMPLING]

'@Test public void yearStart() { Date date = DateUtils.yearStart(); assertNotNull(date); GregorianCalendar calendar = new GregorianCalendar(); calendar.setTime(date); assertEquals(1, calendar.get(DAY_OF_YEAR)); }'

In [53]:
arr_prettify_generated = [prettify_java(lazy_decode(model.decode(input))) for input in arr_generated_input]

In [54]:
arr_prettify_generated

['@Test public void testYearStart() throws Exception {\n     Date date = DateUtils.yearStart();\n     assertNotNull(date);\n     assertEquals(date.getTime(), date.getTime());\n     assertEquals(date.getTime(), date.getTime());\n }\n',
 '@Test public void testYearEnd() {\n     Date date = DateUtils.yearEnd();\n     assertEquals(2015, date.getYear());\n     assertEquals(2015, date.getMonth());\n     assertEquals(2015, date.getMonth());\n     assertEquals(2015, date.getMonth());\n     assertEquals(2015, date.getMonth());\n     assertEquals(2015, date.getMonth());\n     assertEquals(2015, date.getMonth());\n     assertEquals(\n',
 '@Test public void validate_nullClientDataTokenBinding() {\n     tokenBindingValidator.validate(null, null);\n }\n',
 '@Test public void testGetUnsignedShort() {\n     assertEquals(0, UnsignedNumberUtil.getUnsignedShort((byte) 0));\n     assertEquals(0, UnsignedNumberUtil.getUnsignedShort((byte) 0));\n     assertEquals(0, UnsignedNumberUtil.getUnsignedShort((byte

In [58]:
bart_df = pd.DataFrame()

In [59]:
bart_df['ground_truth'] = super_data['test_raw1'].test_raw1.values[:SIZE_SAMPLING]

In [60]:
bart_df['input'] = super_data['test_raw0'].test_raw0.values[:SIZE_SAMPLING]

In [61]:
bart_df['outcome_0'] = arr_prettify_generated

In [62]:
bart_df

,ground_truth,input,outcome_0
0,@Test public void yearStart() { Date date = Da...,DateUtils { public static Date yearStart() { f...,@Test public void testYearStart() throws Excep...
1,@Test public void yearEnd() { Date date = Date...,DateUtils { public static Date yearEnd() { fin...,@Test public void testYearEnd() {\n Date d...
2,@Test void validate_invalid_bindingId_test() {...,TokenBindingValidator { public void validate(T...,@Test public void validate_nullClientDataToken...
3,@Test void getUnsignedShort_test1() { byte[] b...,UnsignedNumberUtil { public static int getUnsi...,@Test public void testGetUnsignedShort() {\n ...
4,@Test void isWithinUnsignedLong_test() { asser...,UnsignedNumberUtil { public static boolean isW...,@Test public void testIsWithinUnsignedLong() {...
...,...,...,...
95,@Test void notEmpty_test_with_array() { Assert...,AssertUtil { public static void notEmpty(Objec...,@Test(expected = IllegalArgumentException.clas...
96,@Test void provide_test() { MetadataStatements...,MetadataStatementsTrustAnchorsProvider impleme...,"@Test void testProvide() {\n Map<AAGUID, S..."
97,@Test void toBytes_BigInteger_uint_max_test() ...,UnsignedNumberUtil { public static byte[] toBy...,@Test public void testToBytes() {\n assert...
98,@Test void deserialize_registration_test(){ Au...,AuthenticationExtensionsAuthenticatorInputs im...,@Test public void testGetCredProtect() throws ...


## Levenshtein evaluation

In [63]:
import textdistance

In [64]:
levenshtein_similarity = textdistance.levenshtein


In [69]:
bart_df['lev_calc'] = [levenshtein_similarity.normalized_similarity(x["ground_truth"].strip(), x["outcome_0"].strip() ) for i,x in bart_df.iterrows()]

In [70]:
bart_df

,ground_truth,input,outcome_0,lev_calc
0,@Test public void yearStart() { Date date = Da...,DateUtils { public static Date yearStart() { f...,@Test public void testYearStart() throws Excep...,0.47
1,@Test public void yearEnd() { Date date = Date...,DateUtils { public static Date yearEnd() { fin...,@Test public void testYearEnd() {\n Date d...,0.40
2,@Test void validate_invalid_bindingId_test() {...,TokenBindingValidator { public void validate(T...,@Test public void validate_nullClientDataToken...,0.21
3,@Test void getUnsignedShort_test1() { byte[] b...,UnsignedNumberUtil { public static int getUnsi...,@Test public void testGetUnsignedShort() {\n ...,0.34
4,@Test void isWithinUnsignedLong_test() { asser...,UnsignedNumberUtil { public static boolean isW...,@Test public void testIsWithinUnsignedLong() {...,0.60
...,...,...,...,...
95,@Test void notEmpty_test_with_array() { Assert...,AssertUtil { public static void notEmpty(Objec...,@Test(expected = IllegalArgumentException.clas...,0.22
96,@Test void provide_test() { MetadataStatements...,MetadataStatementsTrustAnchorsProvider impleme...,"@Test void testProvide() {\n Map<AAGUID, S...",0.29
97,@Test void toBytes_BigInteger_uint_max_test() ...,UnsignedNumberUtil { public static byte[] toBy...,@Test public void testToBytes() {\n assert...,0.28
98,@Test void deserialize_registration_test(){ Au...,AuthenticationExtensionsAuthenticatorInputs im...,@Test public void testGetCredProtect() throws ...,0.24


In [73]:
bart_df['lev_calc'].describe()

count   100.00
mean      0.29
std       0.09
min       0.00
25%       0.24
50%       0.28
75%       0.32
max       0.60
Name: lev_calc, dtype: float64

In [ ]:
def compute_levenshtein_for_samples(tokenizer, df, samples= 30):
    outcomes_df = pd.DataFrame()
    outcomes_df['ground_truth'] = df['ground_truth'].apply(lambda x: x.strip())
    for i in range(0,samples):
        with torch.no_grad():
            outcomes = tokenizer.batch_decode(df[str(i)].to_list(), skip_special_tokens=True)
            outcomes_df['outcome_'+str(i)] = outcomes
            col_name = 'lev_'+str(i)
            outcomes_df[col_name] = [levenshtein_similarity.normalized_similarity(x.strip(), outcomes_df['ground_truth'][step]) for step,x in enumerate(outcomes)]
       
    return outcomes_df

## Bootstrapping

In [77]:
n_bootstraps = 30

In [78]:

#| export
import numpy as np
from statistics import NormalDist

In [ ]:
#| export
def bootstrapping( np_data, np_func, size, flag_clean_nan = False ):
    """
    @size: number of bootstrapping samples
    @np_funct: numpy function for reducing the samples (e.g., median, mean, max)
    @flag_clean_nan: flag to eliminate Nan values in the np tensor
    """
    #Cleaning NaNs
    if flag_clean_nan:
        np_data = np_data[ np.logical_not( np.isnan(np_data) ) ] 
    
    #Creating the boostrap replicates as long as the original data size
    #This strategy might work as imputation 
    bootstrap_repl = [ np_func( np.random.choice( np_data, size=len(np_data) ) ) for i in range( size ) ]
    
    logging.info("Empirical Estimate: " + str(np_func( np_data )) ) #Empirical Mean,Median,Max, etc
    logging.info("Bootstrapped Estimate: " + str( np_func( bootstrap_repl ) ) ) #Bootstrapped Mean,Median,Max, etc
    
    return np.array( bootstrap_repl )